TODO:
- commit
- create 
- creat

In [7]:
import torch 
import torch.nn as nn
import torch.optim as optim
import pytorch_lightning as pl
from torch.utils.data import DataLoader, TensorDataset

import numpy as np
from sklearn.model_selection import train_test_split

In [41]:
class DataModule(pl.LightningDataModule):
    def __init__(self, batch_size):
        super().__init__()
        self.x = np.random.randn(100, 32, 512) # 100 datapoints of 32 consecutive 512-d embeddings
        self.y = np.random.randn(100, 32, 1) # 100 datapoints of 32 consecutive labels
        self.batch_size = batch_size

    def setup(self, stage):
        X_train, X_test, y_train, y_test = train_test_split(self.x, self.y, test_size=0.30, random_state=42)
        X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.50, random_state=42)
        self.train = TensorDataset(torch.from_numpy(X_train).float(), torch.from_numpy(y_train).float())
        self.val = TensorDataset(torch.from_numpy(X_val).float(), torch.from_numpy(y_val).float())
        self.test = TensorDataset(torch.from_numpy(X_test).float(), torch.from_numpy(y_test).float())
    
    def train_dataloader(self):
        return DataLoader(self.train, batch_size=self.batch_size)

    def val_dataloader(self):
        return DataLoader(self.val, batch_size=self.batch_size)

    def test_dataloader(self):
        return DataLoader(self.test, batch_size=self.batch_size)

In [48]:
dm = DataModule(batch_size=40)
dm.setup('fit')
xi, yi = next(iter(dm.train_dataloader()))
xi.shape

#     seq_len = 32
#  batch_size = 40
# hidden_size = 64

torch.Size([40, 32, 512])

In [49]:
SEQ_LEN = 32
BATCH_SIZE = 40
class LipReader(nn.Module):
    def __init__(self, hidden_size, num_layers, dropout_rate, bidirectional):
        super(LipReader, self).__init__()
        num_directions = 2 if bidirectional else 1
        self.gru = nn.GRU(input_size=512, hidden_size=hidden_size, num_layers=num_layers, 
            batch_first=True, bidirectional=bidirectional)
        self.linear = nn.Linear(in_features=num_directions*hidden_size, out_features=1)
        self.sigmoid = nn.Sigmoid()

        # For debugging
        self.num_directions = num_directions
        self.hidden_size = hidden_size
        self.num_layers = num_layers

    def forward(self, embeddings):
        """
        args:
            - embeddings: shape (batch_size, seq_length, 512)
        
        returns:
            - probabilities: shape (batch_size, seq_length, 1)
        """
        output, _ = self.gru(embeddings)
        assert output.shape == (SEQ_LEN, BATCH_SIZE, self.num_directions * self.hidden_size)
        # output from last layer of gru for each t: is (seq_len, batch, num_directions * hidden_size] e.g. (32, 40, 64)
        
        output = torch.transpose(output, 0, 1)
        assert output.shape == (BATCH_SIZE, SEQ_LEN, self.num_directions * self.hidden_size)

        score = self.linear(output)
        assert score.shape == (BATCH_SIZE, SEQ_LEN, 1)

        score = torch.squeeze(score)
        assert score.shape == (BATCH_SIZE, SEQ_LEN)

        probability = self.sigmoid(score)
        assert probability.shape == (BATCH_SIZE, SEQ_LEN)

        return probability

In [ ]:
LipReade